## Iterating the Coffee Cooling Problem: Quenching

In the coffee cooling problem, we used Newton's Law of Cooling to determine the temperature change in the coffee and milk.  One assumption we made was that the temperature of the environment stayed the same even as the coffee cooled: the heat lost by the coffee did not raise the temperature of the environment.

<br> But if the environment is smaller (and so more prone to change temperature), its temperature might change significantly.   We want to iterate on the coffee mixing problem to incorporate this possibility.


## The Physics

The situation we’ll be modeling is the “quenching” of a metal object in a liquid bath.  The very hot object will cool quickly, but in doing so will release thermal energy that will be absorbed by the liquid.  In order to model this, we’ll need two equations, Newton’s law of cooling and a calorimetry equation:

<br>$$\frac{dT}{dt}=r(T-T_{env})$$

$$Q=mC_p ΔT$$

<br> Consider what happens to the thermal energy that leaves the metal object.  Where does it go?  How can we quantify it?  How will that affect the temperature of the liquid?  The major change in the code will be in the change function, but you will also need to adjust the state, system, and results objects in order to deal with the increased number of variables.


✅ ✅  This is a more complex problem than coffee cooling: it happens faster, and contains more drastic changes.   Think about what would happen physically when the hot metal was dropped in water.  What physical occurrences are we ignoring when we use Newton's Law of Cooling?  In other words, what are we abstracting out?

✅ ✅ Answer here.

## Parameters

In order to know the results of our quenching model, we need to know some ey parameters.  We'll use the values of aluminum and water:

<br> Aluminum:  $C_p = 890 ~\frac{J}{kgC} ~~  T_{init} =600 ~C ~~ mass = 0.2 ~kg $

<br>Water:  $C_p = 4186 ~\frac{J}{kgC} ~~  T_{init} = 20 ~C ~~ mass = 1.0 ~kg $

<br>Let's assume as a start that the value of $r$ for the aluminum in this situation is $0.02$.  We can adjust that later.

✅ ✅  A. We can use the code from the Coffee problem to find the change in the temperature of the aluminum in the first time step.  Where does that thermal energy go?

✅ ✅ Answer A here.

✅ ✅ B. Look at the two equations above.  Once we know the change in the temperature of the aluminum, how can we find out how much energy it has lost?

✅ ✅ Answer B here.

✅ ✅ C. Once we know how much energy the aluminum has lost, how can we find the change of temperature in the water?


✅ ✅ Answer C here.

## Part 1: The system

Because our model is more complex, we'll have some new parameters: the material properties of the two materials, and the initial temperature of the water (which will replace `T_env`).  Here is the Coffee versions of `make_system`: adapt it to include the new parameters.

In [ ]:
def make_system(T_init, volume, r, t_end, T_env,T_goal):
    return dict(T_init=T_init, T_final=0, volume=volume,
                  r=r, t_end=t_end, T_env=T_env, T_goal=T_goal,
                  t_0=0,  dt=1)

Now define the parameter values in the cell below, and call `make_system`.  Set `r` to 0.01 for the time being.

In [ ]:
# Define the parameter values


# Call 'make_system()'



## Part 2: The state

First, we want to add a new state variable--the temperature of the water.   Here is the Coffee versions of `make_state`: adapt it so you will now have two state variables.

In [ ]:
def make_state(system):
    return pd.Series(dict(temp=system['T_init']), name='State object')

Now make a state variable, using the system you created in step 1:

In [ ]:
# Call 'make_state' in order to make a state



## Part 3:  The change function


Now we need to make the crucial change: a new change function that changes the function of both the water and the aluminum.  Rewrite this to incorporate the change in temperature of the water, using the equations above.

<br>

You'll need to include three calculations: the change in temperature of the metal, the energy lost by the metal, and then the change in temperature of the water.  You can return two values from a function by separating them by a comma, like this `return dT_al, dT_water`:

In [ ]:
def change_func(t, state, system):
    r, T_env, dt = system['r'], system['T_env'], system['dt']
    deltaT =  -r * (state.temp - T_env) * dt
    return deltaT

In [ ]:
# Now test your change function: the temperature change in the
# aluminum should be negative, but positive for the water
# Notice how we need to have two variables to receive the variables
# returned by 'change_func'
dT_aluminum, dT_water = change_func(0,state,system)
print(dT_aluminum, dT_water)

## Part 4:  Run_simulation


Make adjustments to run simulation.  Remember that `change_func()` returns two values, so you'll need to change how that is called here.  You'll also need two `results` data objects in order to hold the results for both water and aluminum.

In [ ]:
def run_simulation(system, change_func):
    t_array = np.arange(system['t_0'], system['t_end']+1, system['dt'])
    n = len(t_array)
    state = make_state(system)
    results = pd.Series(index=t_array,dtype=object)
    results.iloc[0] = state.temp

    for i in range(n-1):
        t = t_array[i]
        T = state.temp
        delta = change_func(t, state, system)
        state.temp = state.temp + delta
        results.iloc[i+1] = state.temp

    system['T_final'] = results.iloc[-1]
    return results

In [ ]:
# Plot your results and make sure they make sense

results_alum.plot(xlabel='time (s)', ylabel='Temperature (degrees C',
                 title='Temperature of Cooling Object', label='Aluminum', legend=True)
results_water.plot(label='Water', legend=True)

## Part 5 Find real value of `r`

Now see if you can use `root_scalar` to validate a value for `r`.  Let's say we did an experiment, and found that a quenched aluminum piece cooled from 600 to 100 degrees in 40 seconds.  What `r` value would make this happen?

<br> Here's the code from the coffee problem.  You may not need to change much at all!  Just check to see if the names for your parameters have changed.

In [ ]:
def error_func(r, system):
    system['r'] = r
    results = run_simulation(system, change_func)
    return system['T_final'] - system['T_goal']

def find_r(system):
    root_obj = sp.root_scalar(error_func, system, bracket=[0, 1.0])
    return root_obj.root

✅ ✅ Report on the value of `r`.  Use this new value to determine how long it takes for the water and the aluminum to differ in temperature by only 5 degrees C?  You can add to the code to do this get this value, or inspect the data produced by the code you have.

✅ ✅ Answer here.

## Part 5 Analysis

✅ ✅ A.  If we simulated this system (hot object dropped in a liquid) *without* considering the rise in temperature of the liquid, would we underestimate or overestimate the time it took for the object to cool to a given temperature?  Why?

✅ ✅ Answer A here.

✅ ✅ B. Test your answer to question A by using the model.  Can you change 1 parameter value to keep the temperature of the water from changing significantly?

✅ ✅ Report on your results to B here.

✅ ✅ C. One abstraction we've made here is that the heat energy that leaves the metal is immediately spread out into the water.  Does this simplification lead us to over or under estimate the time of cooling?  Why?

✅ ✅ Answer C here.

✅ ✅ D.  In a physical experiment, how might we minimize the effect of temperature differences within the liquid?

✅ ✅ Answer D here.